In [0]:
#Librerías

import openmeteo_requests
import requests_cache
from retry_requests import retry
import requests
import json
import os
import time
import pandas as pd
from io import StringIO
from datetime import datetime, date, timedelta
import numpy as np
import logging

In [0]:
# Logger para avisar

logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(levelname)s | %(message)s', datefmt='%H:%M:%S', force=True)
logger = logging.getLogger("ETL_VOLUMES")

# Paths y carpetas para OpenMeteo

BASE_VOLUME_PATH_METEO = "/Volumes/fire_risk_project/00_landing/meteo_files"
PATH_CLIMA = os.path.join(BASE_VOLUME_PATH_METEO, "open_meteo")

# Directorios

try:
    os.makedirs(PATH_CLIMA, exist_ok=True)
    logger.info("Carpetas configuradas")
except Exception as e:
    logger.warning(f"Aviso: {e}")

In [0]:
# Parametros necesarios para la extracción OpenMeteo

VARIABLES_CLIMA = [
    "temperature_2m", 
    "relative_humidity_2m", 
    "vapour_pressure_deficit", 
    "precipitation", 
    "wind_speed_10m", 
    "wind_direction_10m", 
    "wind_gusts_10m", 
    "et0_fao_evapotranspiration", 
    "soil_moisture_0_to_7cm", 
    "soil_moisture_28_to_100cm", 
    "weather_code", 
    "snow_depth"
]

# Configura el caché y los reintentos 

cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


In [0]:
def etl_clima(year_start, year_end):
    
    df_grid = spark.table("fire_risk_project.00_landing.aux_grid_master").toPandas()
    
    pendientes = []
    for index, row in df_grid.iterrows():
        cell_id = row['cell_id']
        file_name = f"weather_{cell_id}.json"
        final_path = os.path.join(PATH_CLIMA, file_name)
        if not os.path.exists(final_path):
            pendientes.append(row)
            
    if not pendientes:
        logger.info("Todos los archivos ya existen. Nada que descargar.")
        return

    logger.info(f"Descargando {len(pendientes)} puntos")
    
    df_pendientes = pd.DataFrame(pendientes)
    url = "https://archive-api.open-meteo.com/v1/archive"
    
    # Mantenemos lotes de 5, el número más manejable
    CHUNK_SIZE = 5
    
    for i in range(0, len(df_pendientes), CHUNK_SIZE):
        chunk = df_pendientes.iloc[i:i+CHUNK_SIZE]
        
        lats = chunk['latitude'].tolist()
        lons = chunk['longitude'].tolist()
        ids = chunk['cell_id'].tolist()
        
        params = {
            "latitude": lats,
            "longitude": lons,
            "start_date": year_start,
            "end_date": year_end,
            "hourly": VARIABLES_CLIMA
        }
        
        lote_procesado = False
        intentos_lote = 0
        
        while not lote_procesado and intentos_lote < 5:
            try:
                # Llamada a la API
                responses = openmeteo.weather_api(url, params=params)
                
                for idx, response in enumerate(responses):
                    cell_id = ids[idx]
                    lat = response.Latitude()
                    lon = response.Longitude()
                    elev = response.Elevation()
                    
                    hourly = response.Hourly()
                    hourly_data = {
                        "date": pd.date_range(
                            start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
                            end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
                            freq = pd.Timedelta(seconds = hourly.Interval()),
                            inclusive = "left"
                        ).astype(str).tolist()
                    }
                    
                    for var_idx, var_name in enumerate(VARIABLES_CLIMA):
                        var_obj = hourly.Variables(var_idx)
                        if var_obj:
                            values = var_obj.ValuesAsNumpy()
                            hourly_data[var_name] = [None if np.isnan(x) else float(x) for x in values]
                    
                    final_data = {
                        "cell_id": cell_id,
                        "request_latitude": lats[idx],
                        "request_longitude": lons[idx],
                        "elevation_val": elev,
                        "hourly": hourly_data
                    }
                    
                    file_name = f"weather_{cell_id}.json"
                    final_path = os.path.join(PATH_CLIMA, file_name)
                    with open(final_path, "w") as f:
                        json.dump(final_data, f)
                
                logger.info(f"Lote {i//CHUNK_SIZE + 1} procesado correctamente.")
                lote_procesado = True 
                time.sleep(5)
                
            except Exception as e:
                error_msg = str(e)
                # Si el error es por límite de minuto, esperamos 65 segundos y reintentamos
                if "Minutely API request limit" in error_msg or "429" in error_msg:
                    logger.warning(f"Límite por minuto alcanzado en archivo {i}, Lote {i//CHUNK_SIZE + 1}. Reintentando en 65 segundos...")
                    time.sleep(65)
                    intentos_lote += 1
                else:
                    logger.error(f"Error en lote {i}: {e}")
                    break 

logger.info("Función etl_clima lista")

In [0]:
# Ejecución para OpenMeteo

etl_clima("2020-01-01", "2025-12-30")

In [0]:
# Verificación 

try:
    if os.path.exists(PATH_CLIMA):
        # Filtramos solo .json 
        archivos_clima = [f for f in os.listdir(PATH_CLIMA) if f.endswith('.json')]
        archivos_clima.sort()
        count_clima = len(archivos_clima)
        
        total_esperado = 160 # Tamaño de la grilla
        progreso = (count_clima / total_esperado) * 100
        
        print(f"Total : {count_clima} ({progreso:.1f}%)")
        
        if count_clima > 0:
            print(f"Primer celda: {archivos_clima[0]}") 
            print(f"Última celda: {archivos_clima[-1]}") 
            
            # Leemos el último .json para verificar estructura nueva
            ultimo_clima = os.path.join(PATH_CLIMA, archivos_clima[-1])
            with open(ultimo_clima, 'r') as f:
                data = json.load(f)
                
                keys = list(data.get("hourly", {}).keys())
                
                print(f"\nEstructura: {archivos_clima[-1]}")
                print(f"Variables climáticas: {len(keys)}")
                print(f"{keys[:4]}...") # Mostramos 4 para ver si están de verdad
        else:
            print("No hay archivos .json.")
    else:
        print("No existe el directorio.")

except Exception as e:
    print(f"Error en la verificación OpenMeteo: {e}")

In [0]:
# Finalizado proceso de extracción de OpenMeteo por grilla para el período 2020-2025